In [1]:
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [110]:
class Porestats(object):
    def __init__(self, data):
        self.name = data["name"]
        
        self.porosity = data["porosity"]
        self.porosity_sq = self.porosity*self.porosity
        
        #Covariance: C(x) = cov(x)+Vv^2
        self.covariance_x = np.array(data["x"])
        self.covariance_y =  np.array(data["y"])
        self.covariance_z =  np.array(data["z"])
        
        self.covariance_averaged = np.array(data["averaged"])
        
        #Covariance Function: cov(x) = C(x)-Vv^2
        self.covariance_function_x = self.covariance_x-self.porosity_sq
        self.covariance_function_y = self.covariance_y-self.porosity_sq
        self.covariance_function_z = self.covariance_z-self.porosity_sq
        
        self.covariance_function_averaged = self.covariance_averaged-self.porosity_sq
        
        #Normalized Covariance Function: cov(x)/(Vv-Vv^2)
        self.normalized_covariance_x = np.divide(self.covariance_function_x, self.porosity-self.porosity_sq)
        self.normalized_covariance_y = np.divide(self.covariance_function_y, self.porosity-self.porosity_sq)
        self.normalized_covariance_z = np.divide(self.covariance_function_z, self.porosity-self.porosity_sq)
        
        self.normalized_covariance_averaged = np.divide(self.covariance_function_averaged, self.porosity-self.porosity_sq)
        
        self.fit_x = None
        self.fit_y = None
        self.fit_z = None
        
        self.fit_averaged = None
        
        self.poly_x = None
        self.poly_y = None
        self.poly_z = None
        
        self.poly_averaged = None
        
        self.fit_poly()
        
    def fit_poly(self, N=10, res=1):
        self.fit_x = np.polyfit(np.array(range(len(self.covariance_x[0:N])))*res, self.covariance_x[0:N], 1)
        self.poly_x = np.poly1d(self.fit_x)
        
        self.fit_y = np.polyfit(np.array(range(len(self.covariance_y[0:N])))*res, self.covariance_y[0:N], 1)
        self.poly_y = np.poly1d(self.fit_y)
        
        self.fit_z = np.polyfit(np.array(range(len(self.covariance_z[0:N])))*res, self.covariance_z[0:N], 1)
        self.poly_z = np.poly1d(self.fit_z)
        
        self.fit_averaged = np.polyfit(np.array(range(len(self.covariance_averaged[0:N])))*res, self.covariance_averaged[0:N], 1)
        self.poly_averaged = np.poly1d(self.fit_averaged)
        
        self.rc_x = (self.porosity_sq-self.porosity)/self.poly_x[1]
        self.rc_y = (self.porosity_sq-self.porosity)/self.poly_y[1]
        self.rc_z = (self.porosity_sq-self.porosity)/self.poly_z[1]
        
        self.rc_averaged = (self.porosity_sq-self.porosity)/self.poly_averaged[1]
    
    def compute_Sv(self, resolution=1., direc_frac=[1., 1., 1.]):
        self.Sv_i = [-2./direc_frac[0]*self.poly_x[1]*1./resolution, -2./direc_frac[1]*self.poly_y[1]*1./resolution, -2./direc_frac[2]*self.poly_z[1]*1./resolution]
        self.Sv_average = -4*self.poly_averaged[1]*1./resolution
        
    def compute_kozeny_carman_perm(self):
        self.kozeny_i = [(self.porosity**3)/((1-self.porosity)**2)*1./(val**2) for val in self.Sv_i]
        self.kozeny_average = (self.porosity**3)/((1-self.porosity)**2)*1./(self.Sv_average**2) 
        
    def compute_kozeny_constant(self, permeability):
        self.kozeny_c_i = [kozeny/perm for kozeny, perm in zip(self.kozeny_i, permeability)]
        self.kozeny_c_average = self.kozeny_average/np.mean(permeability)

In [137]:
resolution_file = "image_data/resolutions.json"
with open(resolution_file) as data_file:
    resolutions = json.load(data_file)
    
dimensions_file = "image_data/dimensions.json"
with open(dimensions_file) as data_file:
    dimensions = json.load(data_file)
    
permeability_file = "permeability_results/permeability_tensors_edit.json"
with open(permeability_file) as data_file:
    permeability = json.load(data_file)
    
interface_results_dir = "interface_results/"
interface_data = {}

for file in os.listdir(interface_results_dir):
        if file.endswith(".json"):
            with open(interface_results_dir+file) as data_file:
                data = json.load(data_file)
                interface_data[data["name"]] = data

In [122]:
covariance_results_dir = "covariance_results/"
porestats_objs = {}

for file in os.listdir(covariance_results_dir):
        if file.endswith(".json"):
            with open(covariance_results_dir+file) as data_file:
                data = json.load(data_file)
                porestats_objs[data["name"]] = Porestats(data)

In [123]:
for name, covariance_data in porestats_objs.iteritems():
    res = resolutions[str(name)]
    inter = interface_data[str(name)]
    frac = [inter["x_frac"], inter["y_frac"], inter["z_frac"]]
    porestats_objs[name].fit_poly(N=2, res=res)
    perm = permeability[str(name)]
    k = [perm["kx"], perm["ky"], perm["kz"]]
    covariance_data.compute_Sv(direc_frac=frac)
    covariance_data.compute_kozeny_carman_perm()
    covariance_data.compute_kozeny_constant(k)
    print name
    print covariance_data.poly_x[1], covariance_data.poly_y[1], covariance_data.poly_z[1] 
    print covariance_data.poly_averaged[1]
    print covariance_data.rc_x, covariance_data.poly_x[1]
    print covariance_data.Sv_i
    print covariance_data.Sv_average
    print covariance_data.kozeny_i
    print covariance_data.kozeny_average
    print covariance_data.kozeny_c_i
    print covariance_data.kozeny_c_average
    print ""

S9
-3162.67826682 -3645.07014313 -3499.2699052
-3435.67277171
5.45659833504e-05 -3162.67826682
[20636.221080863263, 20463.639457712579, 20752.749810918038]
13742.6910869
[4.2273045759258376e-11, 4.2989078178883904e-11, 4.1799643985787171e-11]
9.53191837835e-11
[15.456323860789169, 20.539454457182945, 22.667919731988707]
42.8593452264

S8
-4318.99784762 -4315.1693323 -5899.20235719
-4844.45651237
5.19425407259e-05 -4318.99784762
[29202.944770816401, 28923.71013381868, 29072.625434887497]
19377.8260495
[1.0556653148504577e-10, 1.0761468672348137e-10, 1.0651506599561799e-10]
2.39755975878e-10
[8.0813390098021713, 7.967327069184968, 8.2340032464145008]
18.2065490719

S3
-2849.15802642 -2858.5258418 -4181.08251219
-3296.25546014
4.91915152842e-05 -2849.15802642
[19775.20139290766, 19884.534870736399, 19706.615217681494]
13185.0218405
[1.7719476950330357e-11, 1.7525154485913978e-11, 1.7843032010353495e-11]
3.98594242494e-11
[123.91242622608641, 41.726558299795187, 163.69754137938986]
177.943

Compute Interfacial area

In [138]:
for name in interface_data.keys():
    print name
    sum_interfaces = interface_data[name]["total_area"]
    interface_area = sum_interfaces*resolutions[name]**2
    print interface_area
    dim = dimensions[name]
    V = (dim**3)*(resolutions[name]**3)
    Sv_count = interface_area/float(V)
    print Sv_count
    porosity = porestats_objs[name].porosity
    kozeny_count = 1./(Sv_count**2)*(porosity**3)/((1-porosity)**2)
    print kozeny_count/permeability[name]["kx"]
    print ""

S9
2.0401295042e-05
19258.5616812
17.7467509716

S8
8.69739849353e-05
27514.7936162
9.10340995289

S3
0.00039304789342
19317.7940578
129.849902274

S2
8.63193380293e-05
26263.3695035
8.8280442639

S1
0.000170334094539
9636.70177145
20.9251908708

S7
6.27473911681e-05
20974.6223425
8.74907650708

S6
4.690326078e-05
13095.7007988
12.2861686294

S5
2.7709492954e-05
16071.720457
12.5398514947

S4
0.000386085132698
19879.0343915
67.7885775148

Isotropic14
7.36376e-06
7363.76
4897.81404741

Isotropic12
1.7774768e-05
17774.768
5.07447650055

Isotropic13
1.3194864e-05
13194.864
9.33980166973

estaillades
0.000918387987563
25293.3876111
70.3251820245

beadpack
5.3902496e-05
53902.496
6.61079632095

bentheimer
0.000540870449926
19962.2893291
11.747512574

C2
0.000189046266573
19343.9429373
484.744338731

C1
5.138218773e-05
34681.5131579
22.6366621816

doddington
0.000549915706755
28160.1812173
43.9764025408

Anisotropic121214
1.5355384e-05
15355.384
13.6886525263

Anisotropic131314
1.1509352e-05

0.000466513139966
17277.2277888


11.6243581182
